In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import qutip as qt 
from bingo.optimizer.tf_adam_optimizer import AdamOptimizer
from bingo.gate_sets import SNAPDispBSGateSet
from bingo.optimizer.GateSynthesizer import GateSynthesizer
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
# device_lib.list_local_devices()
import tensorflow as tf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
N = 25
# We initialize the ECDGateSet here
gate_set_params = {
    'N_snap': 6,
    'N_cav' : N,
    'beta_scale' : 1.0, #maximum |beta| for random initialization 
    'alpha_scale': 1.0 
}
gate_set = SNAPDispBSGateSet(**gate_set_params)

In [5]:
#The target oscillator state.
fock = 1

In [9]:
#the optimization options
synth_params = {
    'gateset' : gate_set,
    'N_blocks':3,
    'N_multistart' : 30, #Batch size (number of circuit optimizations to run in parallel)
    'epochs' : 100, #number of epochs before termination
    'epoch_size' : 10, #number of adam steps per epoch
    'learning_rate' : 0.01, #adam learning rate
    'term_fid' : 0.99, #terminal fidelitiy
    'dfid_stop' : 1e-5, #stop if dfid between two epochs is smaller than this number
    'initial_states' : [qt.tensor(qt.basis(N,0), qt.basis(N,0))], #qubit tensor oscillator, start in |g> |0>
    'target_states' : [qt.tensor(qt.basis(N,fock), qt.basis(N,fock))], #|target>.
    'name' : 'Fock %d' % fock, #name for printing and saving
    'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
    'coherent':False
}
gatesynth = GateSynthesizer(**synth_params)

In [10]:
#create optimization object. 
#initial params will be randomized upon creation
opt = AdamOptimizer(gatesynth)

#print optimization info. this lives in gatesynth, since we eventually want to fully abstract away the optimizer
gatesynth.print_info()

2022-09-14 21:09:37.782420: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x6e17020


N_blocks: 3
optimization_type: state transfer
optimization_masks: {'alphas_rho': None, 'alphas_angle': None, 'betas_rho': None, 'betas_angle': None, 'SNAP_thetas': None, 'BS_phis': None, 'BS_thetas': None}
target_unitary: None
expectation_operators: None
target_expectation_values: None
N_multistart: 30
term_fid: 0.99
dfid_stop: 1e-05
learning_rate: 0.01
epoch_size: 10
epochs: 100
name: Fock 1
filename: None
comment: 
coherent: False
timestamps: []
do_prints: True
filename: Fock 1.h5

Best circuit parameters found:
alphas:    tf.Tensor([-0.11908-0.10995j -0.05424-0.01009j -0.00226+0.00325j], shape=(3,), dtype=complex64)
betas:    tf.Tensor([-0.81613+0.45094j -0.04117-0.28139j  0.73956-0.54276j], shape=(3,), dtype=complex64)
SNAP_thetas:    tf.Tensor(
[[-2.52406 -2.43319  0.52586 -3.03178  2.82216 -2.73693]
 [ 2.18138 -0.07154  0.9486   0.05455 -0.93868 -0.42909]
 [ 2.97029  0.85912  0.86434 -2.84794 -1.67799 -0.47358]], shape=(3, 6), dtype=float32)
BS_phis:    tf.Tensor([-1.70689  0.756

In [11]:
#run optimizer.
#note the optimizer can be stopped at any time by interrupting the python consle,
#and the optimization results will still be saved and part of the opt object.
#This allows you to stop the optimization whenever you want and still use the result.
opt.optimize()

Start time: 2022-09-14 21:09:41
 Epoch: 10 / 100 Max Fid: 0.993900 Avg Fid: 0.735203 Max dFid: 0.169293 Avg dFid: 0.041072 Elapsed time: 0:01:09.841342 Expected remaining time: 0:10:28.572076 (real part)

 Optimization stopped. Term fidelity reached.

N_blocks: 3
optimization_type: state transfer
optimization_masks: {'alphas_rho': None, 'alphas_angle': None, 'betas_rho': None, 'betas_angle': None, 'SNAP_thetas': None, 'BS_phis': None, 'BS_thetas': None}
target_unitary: None
expectation_operators: None
target_expectation_values: None
N_multistart: 30
term_fid: 0.99
dfid_stop: 1e-05
learning_rate: 0.01
epoch_size: 10
epochs: 100
name: Fock 1
filename: None
comment: 
coherent: False
timestamps: ['2022-09-14 21:09:41']
do_prints: True
filename: Fock 1.h5

Best circuit parameters found:
alphas:    tf.Tensor([ 0.4818 -0.6112j  -0.62704+0.88441j  0.15721-0.14647j], shape=(3,), dtype=complex64)
betas:    tf.Tensor([-0.29269-0.30345j -0.21321-0.02248j  0.49765+0.47597j], shape=(3,), dtype=compl

'2022-09-14 21:09:41'

In [12]:
#can print info, including the best circuit found.
gatesynth.print_info() 

N_blocks: 3
optimization_type: state transfer
optimization_masks: {'alphas_rho': None, 'alphas_angle': None, 'betas_rho': None, 'betas_angle': None, 'SNAP_thetas': None, 'BS_phis': None, 'BS_thetas': None}
target_unitary: None
expectation_operators: None
target_expectation_values: None
N_multistart: 30
term_fid: 0.99
dfid_stop: 1e-05
learning_rate: 0.01
epoch_size: 10
epochs: 100
name: Fock 1
filename: None
comment: 
coherent: False
timestamps: ['2022-09-14 21:09:41']
do_prints: True
filename: Fock 1.h5

Best circuit parameters found:
alphas:    tf.Tensor([ 0.4818 -0.6112j  -0.62704+0.88441j  0.15721-0.14647j], shape=(3,), dtype=complex64)
betas:    tf.Tensor([-0.29269-0.30345j -0.21321-0.02248j  0.49765+0.47597j], shape=(3,), dtype=complex64)
SNAP_thetas:    tf.Tensor(
[[ 2.89539 -0.36044  0.08598  1.56052  0.98199 -1.21803]
 [ 3.13462  1.94719 -2.74682  1.91634  0.20266 -1.342  ]
 [ 0.36181  1.13246 -1.24795  1.91989  2.75545  2.00178]], shape=(3, 6), dtype=float32)
BS_phis:    tf.Te

In [25]:
print(gatesynth.best_fidelity())

0.963334


In [26]:
#can also get the best circuit parameters directly, could save this to a .npz file.
best_circuit =  gatesynth.best_circuit()
betas = best_circuit['betas']
phis = best_circuit['phis']
thetas = best_circuit['thetas']
print(best_circuit)

KeyError: 'betas'